In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect("./sqlite_db_pythonsqlite.db")

In [4]:
cur = conn.cursor()

In [12]:
qry10 = """SELECT sub.name, SUM(sub.inner_total_revenue) AS total_revenue
FROM
(SELECT Facilities.name, SUM(Facilities.guestcost * Bookings.slots) AS inner_total_revenue
FROM Bookings
INNER JOIN Facilities ON Facilities.facid = Bookings.facid
INNER JOIN Members ON Members.memid = Bookings.memid
WHERE Members.memid = 0
GROUP BY Facilities.name
UNION ALL
SELECT Facilities.name, SUM(Facilities.membercost * Bookings.slots) AS total_revenue
FROM Bookings
INNER JOIN Facilities ON Facilities.facid = Bookings.facid
INNER JOIN Members ON Members.memid = Bookings.memid
WHERE Members.memid > 0
GROUP BY Facilities.name) AS sub
GROUP BY sub.name
HAVING SUM(sub.inner_total_revenue) < 1000
ORDER BY SUM(sub.inner_total_revenue)"""

cur.execute(qry10)
results = cur.fetchall()
print(pd.DataFrame(results, columns=['Facility','Revenue']))

        Facility  Revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [11]:
qry11 = """SELECT M1.surname || ", " || M1.firstname AS member_name, 
CASE WHEN M1.recommendedby > 0 THEN M2.surname || ", " || M2.firstname
ELSE "None" END AS reco_by
FROM Members AS M1
INNER JOIN Members AS M2 ON M2.memid = M1.recommendedby
WHERE M1.memid > 0
ORDER BY member_name"""

cur.execute(qry11)
results = cur.fetchall()
print(pd.DataFrame(results, columns=['Member','Recommended By']))

                      Member      Recommended By
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

In [15]:
qry12 = """SELECT Members.surname || ", " || Members.firstname AS member_name, Facilities.name, ROUND(SUM(Bookings.slots)/2,1) AS hours_use
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Facilities.facid = Bookings.facid
WHERE Members.memid > 0
GROUP BY Members.memid, Facilities.facid
ORDER BY member_name, Facilities.name"""

cur.execute(qry12)
results = cur.fetchall()
print(pd.DataFrame(results, columns=['Member','Facility', 'Hours of Use']))

                       Member         Facility  Hours of Use
0             Bader, Florence  Badminton Court          13.0
1             Bader, Florence   Massage Room 2           2.0
2             Bader, Florence       Pool Table          11.0
3             Bader, Florence    Snooker Table          33.0
4             Bader, Florence     Squash Court           2.0
..                        ...              ...           ...
197      Tupperware, Hyacinth     Squash Court           1.0
198  Worthington-Smyth, Henry  Badminton Court           7.0
199  Worthington-Smyth, Henry   Massage Room 1           1.0
200  Worthington-Smyth, Henry       Pool Table          18.0
201  Worthington-Smyth, Henry     Table Tennis           3.0

[202 rows x 3 columns]


In [18]:
qry13 = """SELECT Facilities.name, strftime('%m', Bookings.starttime) AS month, sum(Bookings.slots)/2 AS hours_use
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Facilities.facid = Bookings.facid
WHERE Members.memid > 0
GROUP BY Facilities.facid, strftime('%m', Bookings.starttime)
ORDER BY Facilities.name, month DESC"""

cur.execute(qry13)
results = cur.fetchall()
print(pd.DataFrame(results, columns=['Facility', 'Month', 'Hours of Use']))

           Facility Month  Hours of Use
0   Badminton Court    09           253
1   Badminton Court    08           207
2   Badminton Court    07            82
3    Massage Room 1    09           201
4    Massage Room 1    08           158
5    Massage Room 1    07            83
6    Massage Room 2    09            14
7    Massage Room 2    08             9
8    Massage Room 2    07             4
9        Pool Table    09           221
10       Pool Table    08           151
11       Pool Table    07            55
12    Snooker Table    09           202
13    Snooker Table    08           158
14    Snooker Table    07            70
15     Squash Court    09            92
16     Squash Court    08            92
17     Squash Court    07            25
18     Table Tennis    09           200
19     Table Tennis    08           148
20     Table Tennis    07            49
21   Tennis Court 1    09           208
22   Tennis Court 1    08           169
23   Tennis Court 1    07           100
